In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_excel("/Users/fikrialinfijar/Desktop/Perkuliahan/Python Notebook/OnlineRetail.xlsx")
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


In [5]:
# df['CustomerID'] = df['CustomerID'].astype(int)
# df.info()

In [6]:
# Menghitung jumlah missing value pada setiap atribut
missing_values = df.isnull().sum()

# Menampilkan jumlah missing value pada setiap atribut
print("Jumlah missing value pada setiap atribut:")
print(missing_values)

Jumlah missing value pada setiap atribut:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [7]:
# Mengganti missing value pada atribut 'CustomerID' dengan nilai acak dari nilai yang ada
existing_customer_ids = df['CustomerID'].dropna().unique()
random_customer_id = np.random.choice(existing_customer_ids)
df['CustomerID'].fillna(random_customer_id, inplace=True)

# Menghilangkan baris dengan missing value pada atribut 'Description'
df.dropna(subset=['Description'], inplace=True)

# Menghilangkan baris dengan nilai negatif pada atribut 'Quantity'
df = df[df['Quantity'] >= 0]

# Menghilangkan baris dengan nilai 0 pada atribut 'UnitPrice'
df = df[df['UnitPrice'] > 0]
print (df.isnull(),sum)
print("Jumlah baris setelah preprocessing", len(df))

        InvoiceNo  StockCode  Description  Quantity  InvoiceDate  UnitPrice   
0           False      False        False     False        False      False  \
1           False      False        False     False        False      False   
2           False      False        False     False        False      False   
3           False      False        False     False        False      False   
4           False      False        False     False        False      False   
...           ...        ...          ...       ...          ...        ...   
541904      False      False        False     False        False      False   
541905      False      False        False     False        False      False   
541906      False      False        False     False        False      False   
541907      False      False        False     False        False      False   
541908      False      False        False     False        False      False   

        CustomerID  Country  
0            False   

In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['YearMonth'] = df['InvoiceDate'].dt.strftime("%Y-%m")
df['YearMonth'] = pd.to_datetime(df['YearMonth'])
df['CohortMonth'] = df.groupby(by=['CustomerID'])['YearMonth'].transform('min')

def calculate_cohort_index(df, transactions_col, cohort_month):
    
    year_diff = df[transactions_col].dt.year - df[cohort_month].dt.year

    month_diff = df[transactions_col].dt.month - df[cohort_month].dt.month
    
    cohort_index = year_diff * 12 + month_diff + 1
    
    return cohort_index

df['CohortIndex'] = calculate_cohort_index(df, 'InvoiceDate', 'CohortMonth')
df['CohortIndex'].max()
df.tail()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,YearMonth,CohortMonth,CohortIndex
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011-12-01,2011-08-01,5
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-01,2011-08-01,5
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-01,2011-08-01,5
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-01,2011-08-01,5
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-01,2011-08-01,5


In [9]:
users_in_cohorts = df.groupby(by=['CohortMonth', 'CohortIndex'])['CustomerID'].nunique()
users_in_cohorts = users_in_cohorts.reset_index()
users_in_cohorts.head()


,CohortMonth,CohortIndex,CustomerID
0,2010-12-01,1,886
1,2010-12-01,2,325
2,2010-12-01,3,287
3,2010-12-01,4,341
4,2010-12-01,5,322


In [10]:
cohorts_fin = users_in_cohorts.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')
cohorts_fin

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,886.0,325.0,287.0,341.0,322.0,353.0,322.0,310.0,314.0,351.0,332.0,446.0,236.0
2011-01-01,417.0,92.0,111.0,96.0,134.0,120.0,103.0,101.0,125.0,136.0,152.0,49.0,NaN
2011-02-01,380.0,71.0,71.0,108.0,103.0,94.0,96.0,106.0,94.0,116.0,26.0,NaN,NaN
2011-03-01,452.0,68.0,114.0,90.0,101.0,76.0,121.0,104.0,126.0,39.0,NaN,NaN,NaN
2011-04-01,300.0,64.0,61.0,63.0,59.0,68.0,65.0,78.0,22.0,NaN,NaN,NaN,NaN
2011-05-01,284.0,54.0,49.0,49.0,59.0,66.0,75.0,27.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,242.0,42.0,38.0,64.0,56.0,81.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,188.0,34.0,39.0,42.0,51.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,169.0,35.0,42.0,41.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
cohort_sizes = cohorts_fin.iloc[:, 0]
retention = cohorts_fin.divide(cohort_sizes, axis='rows').round(3)
retention

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2010-12-01,1.0,0.367,0.324,0.385,0.363,0.398,0.363,0.350,0.354,0.396,0.375,0.503,0.266
2011-01-01,1.0,0.221,0.266,0.230,0.321,0.288,0.247,0.242,0.300,0.326,0.365,0.118,NaN
2011-02-01,1.0,0.187,0.187,0.284,0.271,0.247,0.253,0.279,0.247,0.305,0.068,NaN,NaN
2011-03-01,1.0,0.150,0.252,0.199,0.223,0.168,0.268,0.230,0.279,0.086,NaN,NaN,NaN
2011-04-01,1.0,0.213,0.203,0.210,0.197,0.227,0.217,0.260,0.073,NaN,NaN,NaN,NaN
2011-05-01,1.0,0.190,0.173,0.173,0.208,0.232,0.264,0.095,NaN,NaN,NaN,NaN,NaN
2011-06-01,1.0,0.174,0.157,0.264,0.231,0.335,0.095,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,1.0,0.181,0.207,0.223,0.271,0.112,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,1.0,0.207,0.249,0.243,0.124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df['PurchaseSize'] = df['Quantity']*df['UnitPrice']
# calculate monthly spend per customer
monthly_revenue = df.groupby(by=['CustomerID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue


816.3695502831777

In [14]:
#mengasumsikan lamanya pelanggan bersama dengan perusahaan adalah 3 thun atau 36 bulan
lifespan_months = 36
CLV_basic = monthly_revenue * lifespan_months
CLV_basic

29389.303810194397

In [17]:
# first mean() averages the revenue per invoice 
# second mean() averages the revenue per purchase
revenue_per_purchase = df.groupby(by=['InvoiceNo'])['PurchaseSize'].mean().mean()
revenue_per_purchase

64.87696679155891

In [18]:
# calculate average number of unique invoices per customer each month
avg_frequency_per_month = df.groupby(by=['CustomerID', 'YearMonth'])['InvoiceNo'].nunique().mean()
avg_frequency_per_month


1.5276289606612583

In [19]:
CLV_granular = revenue_per_purchase * avg_frequency_per_month * lifespan_months
CLV_granular


3567.885600623188

In [20]:
# calculate monthly spend per customer
monthly_revenue = df.groupby(by=['CustomerID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

816.3695502831777

In [21]:
#calculate average retention rate per customer
avg_retention = retention.iloc[:, 1:].mean().mean()
avg_retention


0.25147696308321305

In [22]:
avg_churn = 1 - avg_retention
avg_churn

0.748523036916787

In [23]:
CLV_traditional = monthly_revenue * (avg_retention/avg_churn)
CLV_traditional

274.2709644641769

In [24]:
print("Average CLV: {:.2f} USD at {:.1%} average retention rate and {:.2f} USD average monthly revenue".format(CLV_traditional, avg_retention, monthly_revenue))


Average CLV: 274.27 USD at 25.1% average retention rate and 816.37 USD average monthly revenue
